## import statments

In [1]:
import os
import numpy as np
import librosa
import shutil
import soundfile as sf
import tensorflow as tf
from tensorflow.keras import layers, models

## Load Files and set output

In [2]:
input_sound_directory = "./input"
output_dataset_directory = "./output"

## Function to load and preprocess audio files

In [3]:

def load_audio_files(sound_directory, sr=16000):
    audio_data = []
    for filename in os.listdir(sound_directory):
        if filename == '.DS_Store':
            continue  # Skip .DS_Store file
        file_path = os.path.join(sound_directory, filename)
        audio, _ = librosa.load(file_path, sr=sr)
        audio_data.append(audio)
    return audio_data


## Function to create the SampleRNN model

In [4]:

def create_samplernn_model():
    model = models.Sequential()
    model.add(layers.LSTM(256, input_shape=(None, 1), return_sequences=True))
    model.add(layers.LSTM(256, return_sequences=True))
    model.add(layers.TimeDistributed(layers.Dense(1)))
    return model


## Function to generate synthetic sound using the SampleRNN model

In [5]:

def generate_synthetic_sound(model, input_audio):
    input_audio = np.expand_dims(input_audio, axis=-1)
    synthetic_sound = model.predict(np.expand_dims(input_audio, axis=0))
    return np.squeeze(synthetic_sound, axis=0)

## Simple Main Function without hyperparemeter

In [ ]:

if __name__ == "__main__":
    # Load the raw audio files
    raw_audio_data = load_audio_files(input_sound_directory)
    
     # Remove the output folder if it already exists
    if os.path.exists(output_dataset_directory):
        shutil.rmtree(output_dataset_directory)

    # Create a new output folder
    os.makedirs(output_dataset_directory)

    # Create and compile the SampleRNN model
    model = create_samplernn_model()
    model.compile(optimizer="adam", loss="mse")

    # Generate synthetic sound datasets for each input file
    for idx, raw_audio in enumerate(raw_audio_data):
        synthetic_datasets = []
        for _ in range(3):  # Generate 3 unique synthetic datasets
            synthetic_sound = generate_synthetic_sound(model, raw_audio)
            synthetic_datasets.append(synthetic_sound)

        # Save the synthetic datasets to the output directory
        os.makedirs(output_dataset_directory, exist_ok=True)
        for idx, synthetic_sound in enumerate(synthetic_datasets):
            output_filename = f"synthetic_sound_dataset_{idx}.wav"
            output_path = os.path.join(output_dataset_directory, output_filename)
            sf.write(output_path, synthetic_sound, samplerate=16000)

2023-08-02 20:18:05.510776: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1/1 [==============================] - 85s 85s/step
